<a href="https://colab.research.google.com/github/sucrammal/vectari/blob/main/COSTAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install tqdm

In [ ]:
!pip install ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import openai
from openai import AzureOpenAI
from tqdm import tqdm
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

import ipdb

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import *

nltk.download('punkt')
nltk.download('stopwords')

from google.colab import drive
drive.mount('/content/drive')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive


In [ ]:
from google.colab import userdata

# Setting up API
gpt_config = {
    'model': 'gpt-4-vectari-1106preview',
    'azure_endpoint': 'https://oai-vectari-eastus2.openai.azure.com/',
    'api_key': userdata.get('openAPIKey'),
    'api_version': '2023-12-01-preview'
}

# gpt_config = {
#     'model': 'gpt-35-turbo',
#     'azure_endpoint': 'https://oai-vectari-northcentral-us.openai.azure.com/',
#     'api_key': userdata.get('openAPIKey3.5'),
#     'api_version': '2024-02-15-preview'
# }

print(gpt_config['azure_endpoint'])

client = AzureOpenAI(
  azure_endpoint = gpt_config['azure_endpoint'],
  api_key = gpt_config['api_key'],
  api_version = gpt_config['api_version']
)

https://oai-vectari-eastus2.openai.azure.com/


**Load data**

In [ ]:
# instantiate input file from scratch.
input_file = input("Enter file path of raw dataset")
with open(input_file, 'r') as file:
  all_complaints = pd.read_csv(file)

# extract complaints with transcripts
complaints_with_transcripts = all_complaints.loc[all_complaints["Consumer complaint narrative"].notna()]
complaints_with_transcripts.shape[0]

In [ ]:
# load LLM pre-cleaned transcripts from csv
input_file = input("Enter a file name that exists in the Data folder: ")
cleaned_dataset = pd.DataFrame()

with open(input_file, 'r') as file:
    cleaned_dataset = pd.read_csv(file)

Enter a file name that exists in the Data folder: /content/drive/MyDrive/Internships and Work/Summer 2024/Vectari/out/LLM_cleaned.csv


**This function uses the LLM to clean incoming transcripts**

In [ ]:
cleaning_system = """
CONTEXT: You will receive a transcript of a phone call between a call center agent at a mortgage servicing company and a customer/borrower. You need to clean this transcript.

#############

OBJECTIVE: Your first job is to clean the incoming low-quality transcripts from the first dataset. These are calls about financial services and we have seen some text completely out of place.  For instance 'the first drug was fixed, and any drug after that was gonna be viral' should pretty clearly be
'the first rate was fixed, and any rate after that was going to be variable'.
Your job is to parse the text and using your natural language understanding and contextual awareness, please re-write the transcript and clean any grammar / things that do not make sense in financial use cases.
Additionally, remove any unwanted 'x's, 'xxxx's, any other similar phrases representing redaction, and simply any text or numbers that aren't words. Remove any dates, names, and otherwise sensitive information. After the response is generated, remove all full stops, commas, semicolons, colons, and quotation marks.

#############

# STYLE #
Financial complaint transcript

#############

# TONE #
Professional, technical

#############

# AUDIENCE #
Banks. I want accurate classification so that they will adopt this methodology to help organize their complaints database.

#############

# RESPONSE: A PARAGRAPH OF TEXT #
"""

**Then, applying the cleaning function, ask the LLM to cluster the data**

In [ ]:
def clean_transcript(transcript):
  messages = [
    {
        "role":"system",
        "content": cleaning_system
    },
    {
        "role":"user",
        "content":f'"""{transcript}"""'
    }
  ]

  response = client.chat.completions.create(
      model=gpt_config['model'],
      messages = messages
  )

  LLM_cleaned_sample = response.choices[0].message.content
  return LLM_cleaned_sample

In [ ]:
cluster_system = """
CONTEXT: You will now be recieving a large text containing transcripts of phone calls between a call center agent at a mortgage servicing company and a customer/borrower. This large text represents a CLUSTER of transcripts that have been categorized based on content under some label, for example, "fraud or scam".
This label, referred to as the [OFFICIAL_CATEGORY_NAME], will be at the top of the inputted prompt before the transcripts. After a newline character, individual transcripts will be consecutively shown, separated by two newline characters from one another.

#############

OBJECTIVE:
Based on the cluster you have received,
OFFICIAL CATEGORY NAME: Restate the OFFICIAL_CATEGORY_NAME given on the top of the inputted large text.
CLUSTER_INFORMATION: Describe the cluster based on the content of the transcript data.

#############

STYLE:
Business analytics report

#############

TONE:
Professional, technical

#############

AUDIENCE:
Banks. I want to help organize their complaints database.

#############

RESPONSE: MARKDOWN REPORT FORMATED IN THE FOLLOWING STYLE:
– Offical name: [OFFICIAL_CATEGORY_NAME]
— Complaint Group Profile: [CLUSTER_INFORMATION]

"""

In [ ]:
def cluster_complaints(cleaned_dataset):
    clusters = cleaned_dataset.groupby('Issue')
    cluster_info = {}

    for label, group in tqdm(clusters, desc="Creating clusters"):
        # Collect all transcripts in this cluster
        transcripts = group['Consumer complaint narrative'].tolist()

        # Weird error. Passing the full, unsplit dataset through this function worked.
        # But, passing a train set (from train_test_split) would crash since a value in transcripts would somehow not be a string.
        filtered_list = [s for s in transcripts if isinstance(s, str)]

        # Extract keywords and name cluster group.
        all_transcripts_joined = "\n\n".join(filtered_list)
        messages = [
          {
              "role":"system",
              "content": cluster_system
          },
          {
              "role":"user",
              "content":f'""" Official name: {label} \n Transcrips: \n {all_transcripts_joined}"""'
          }
        ]

        response = client.chat.completions.create(
            model=gpt_config['model'],
            messages = messages
        )

        cluster_report = response.choices[0].message.content


        cluster_info[label] = cluster_report

    return cluster_info

In [ ]:
classify_system = """
CONTEXT: You have previously created Complaint Groups of transcripts of complaint phone calls between a call center agent at a mortgage servicing company and a customer/borrower based
on the category of the complaints. Transcripts under each Complaint Group has been categorized based on content under some label, for example, "fraud or scam".

You will now be recieving three things: the first is a set of markdown reports representing the information you have gathered and organized for each CLUSTER, and are of the following format:
"– Offical name: [OFFICIAL_CATEGORY_NAME]
— Complaint Group Profile: [CLUSTER_INFORMATION]"
Each markdown report for each Complaint Group will be separated by a new line character, "\n".

The second piece of data you will receive is a transcript that has not been categorized into one of your CLUSTERs yet.

The third piece of data you will receive is a comma-separated list of [OFFICIAL_CATEGORY_NAME]'s, representing all possible categories that the aformentioned transcript can belong to.

#############

OBJECTIVE: Based on your natural language understanding, and your understanding of the cluster markdown reports, determine which [OFFICIAL_CATEGORY_NAME] from the list, the incoming transcript belongs under.

#############

STYLE: Business analytics report

#############

TONE: Professional, technical

#############

AUDIENCE: Banks. I want accurate classification so that they will adopt this methodology to help organize their complaints database.

#############

# RESPONSE: Single line of text containing ONLY one [OFFICIAL_CATEGORY_NAME] of your choosing from the comma-separated category name list: #
[OFFICIAL_CATEGORY_NAME]
"""

In [ ]:
def classify_transcript(cluster_info, new_transcript):
  # unpack official label names,
  labels = cluster_info.keys()

  # Join the keys into a string separated by commas
  labels_string = ', '.join(labels)

  # unpack and format cluster_info, aka the reports
  markdown_reports = ""

  for report in cluster_info.values():
    markdown_reports = "\n".join(report)

  messages = [
          {
              "role":"system",
              "content": classify_system
          },
          {
              "role":"user",
              "content":f'"""Cluster markdown reports: \n {markdown_reports} \n Transcript to-be classified: \n {new_transcript} \n Category list: \n {labels_string}"""'
          }
        ]

  response = client.chat.completions.create(
            model=gpt_config['model'],
            messages = messages
        )

  complaint_label = response.choices[0].message.content

  return complaint_label


In [ ]:
# Run this to clean transcipts from scratch.
LLM_cleaned_sentences = []
all_labels = complaints_with_transcripts['Issue']

# First, LLM cleanup
tqdm.pandas(desc="LLM transcript cleaning")
LLM_cleaned_sentences = complaints_with_transcripts["Consumer complaint narrative"].progress_apply(lambda x : clean_transcript(x))

cleaned_dataset = pd.DataFrame({'Consumer complaint narrative': LLM_cleaned_sentences, 'Issue': all_labels})

**Train-test split before clustering**

In [ ]:
train_dataset, test_dataset = train_test_split(cleaned_dataset, random_state=104, test_size=0.20, shuffle = True)

In [ ]:
cluster_info = cluster_complaints(train_dataset)

Creating clusters: 100%|██████████| 6/6 [02:47<00:00, 27.98s/it]


In [ ]:
print(cluster_info.keys())

dict_keys(['Fraud or scam', 'Incorrect/missing disclosures or info', 'Money was not available when promised', 'Other service issues', 'Other transaction issues', 'Wrong amount charged or received'])


In [ ]:
# Optional: export cluster data to a file.
import json

# Convert dictionary to a string
dict_string = json.dumps(cluster_info, indent=4)

# Write the string to a file
with open('cluster_info.txt', 'w') as file:
    file.write(dict_string)

!cp cluster_info.txt "/content/drive/MyDrive/Internships and Work/Summer 2024/Vectari/out"

In [ ]:
print(test_dataset.iloc[0])
classify_transcript(cluster_info, test_dataset.iloc[0]['Consumer complaint narrative'])

Consumer complaint narrative    On XXXX/XXXX/15, I initiated a money transfer ...
Issue                                                    Other transaction issues
Name: 963, dtype: object


'Wrong amount charged or received'

In [ ]:
predicted_labels = []
for transcript in tqdm(test_dataset['Consumer complaint narrative'], desc="Predicting labels"):
    predicted_labels.append(classify_transcript(cluster_info, transcript))

Predicting labels: 100%|██████████| 300/300 [31:28<00:00,  6.29s/it]


In [ ]:
test_labels = test_dataset['Issue'].tolist()

# Evaluation
print("Accuracy:", accuracy_score(test_labels, predicted_labels))
print("\nClassification Report:\n", classification_report(test_labels, predicted_labels))

Accuracy: 0.25333333333333335

Classification Report:
                                        precision    recall  f1-score   support

                        Fraud or scam       0.83      0.55      0.66       115
Incorrect/missing disclosures or info       0.00      0.00      0.00        11
Money was not available when promised       0.00      0.00      0.00        44
                 Other service issues       0.00      0.00      0.00        25
             Other transaction issues       0.00      0.00      0.00        92
     Wrong amount charged or received       0.06      1.00      0.11        13

                             accuracy                           0.25       300
                            macro avg       0.15      0.26      0.13       300
                         weighted avg       0.32      0.25      0.26       300



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
